In [2]:
class Nerdle
#nullable enable
{
    public required (char? mandatory, char[]? forbiden)[] Slot { get; init; }
    private readonly (char symbol, int qty, int min)[] _symbols = default!;
    public required (char symbol, int qty, int min)[] Symbols
    {
        get => _symbols;
        init => _symbols = value
            .Where(static s => s is (_, > 0, _) or (_, _, >= 0))
            .Where(((Slot?.Any(static s => s is ('=', _))) ?? false) ? static s => s is (_, _, _) : static _ => true)
            .ToArray();
    }

    public IEnumerable<string> GetAllLines(bool printMaxCombinatory = false)
    {
        // System.Diagnostics.Debugger.Launch();
        var symbols = Enumerable.Repeat((Slot, Symbols), Slot.Length)
            .Select(static (p, pos) => p.Slot[pos] switch
            {
                (char m, null or []) => Enumerable.Repeat(m, 1),
                (null, char[] f) => p.Symbols.Select(static s => s.symbol).Except(f),
                _ => throw new System.Diagnostics.UnreachableException(),
            })
            .Select(Enumerable.ToArray)
            .ToArray();

        if (printMaxCombinatory)
        {
            var combinatory = 1L;
            for (var i = 0; i < symbols.Length; i++)
            {
                Console.Error.WriteLine($"[{i}]: {symbols[i].Length}");
                combinatory *= symbols[i].Length;
            }
            Console.Error.WriteLine($"Max combinatory: {combinatory}");
        }

        return Process(0, new char[Slot.Length], symbols).Where(IsValidLine);

        static IEnumerable<string> Process(int pos, char[] line, char[][] symbols)
        {
            foreach (var symbol in symbols[pos])
            {
                line[pos] = symbol;
                if (pos < line.Length - 1)
                    foreach (var l in Process(pos + 1, line, symbols))
                        yield return l;
                else
                    yield return new(line);
            }
        }

        bool IsValidLine(string line)
        {
            if (line == "8+()-(91=2")
                System.Diagnostics.Debugger.Break();

            return line.IndexOf('=') is var index and >= 3
                    && CheckSymbolOnTheRight(line, index)
                    && CheckFirstAndLast(line)
                    && CheckBeforeAndAfterOperator(line, index)
                    && CheckSymbolQty(line, Symbols)
                    && CheckParenthesis(line, index)
                    && CheckExponent(line, index);

            static bool CheckSymbolOnTheRight(string line, int index)
            {
                var a = line.IndexOf('+', index + 1);
                var s = line.IndexOf('-', index + 1);
                var m = line.IndexOf('*', index + 1);
                var d = line.IndexOf('/', index + 1);
                var e = line.IndexOf('=', index + 1);
                var l = line.IndexOf('(', index + 1);
                var r = line.IndexOf(')', index + 1);
                var q = line.IndexOf('²', index + 1);
                var c = line.IndexOf('³', index + 1);
                return ! (a != -1 || s != -1 || m != -1 || d != -1 || e != -1 || l != -1 || r != -1 || q != -1 || c != -1);
            }

            static bool CheckFirstAndLast(string line)
            => line[0] is >= '0' and <= '9' or '(' && line[^1] is >= '0' and <= '9';

            static bool CheckBeforeAndAfterOperator(string line, int index)
            {
                for (var i = 1; i < index; i++)
                    if (line[i] is '+' or '-' or '*' or '/' && !(line[i - 1] is >= '0' and <= '9' or ')' && line[i + 1] is >= '0' and <= '9' or '('))
                        return false;
                return true;
            }

            static bool CheckSymbolQty(string line, (char symbol, int qty, int min)[] symbols)
            {
                foreach (var symbol in symbols)
                    switch (symbol)
                    {
                        case (_, 0, 0):
                            break;
                        case (_, <= 0, var min):
                            if (line.Count(symbol.symbol.Equals) < min)
                                return false;
                            break;
                        case (_, var qty, _):
                            if (line.Count(symbol.symbol.Equals) != qty)
                                return false;
                            break;
                    }
                return true;
            }

            static bool CheckParenthesis(string line, int index)
            {
                var depth = 0;
                for (var i = 0; i < index; i++)
                {
                    var c = line[i];
                    if (c is '(')
                    {
                        depth++;
                        if ((i >=1 && line[i - 1] is not ('+' or '-' or '*' or '/')) || line[i + 1] is ')')
                            return false;
                    }
                    else if (c is ')')
                    {
                        depth--;
                        if (line[i + 1] is not ('+' or '-' or '*' or '/' or '²' or '³'))
                            return false;
                    }
                    if (depth < 0)
                        return false;
                }
                return depth == 0;
            }

            static bool CheckExponent(string line, int index)
            {
                for (var i = 1; i < index; i++)
                    if (line[i] is '²' or '³' && line[i - 1] is not (>= '0' and <= '9' or ')'))
                        return false;
                return true;
            }
        }
    }
}

In [3]:
var normal = new Nerdle() // first input = 12+57=69
#nullable enable
{
    Slot = new (char?, char[]?)[8]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('1', 0, 0),
        ('2', 0, 0),
        ('3', 0, 0),
        ('4', 0, 0),
        ('5', 0, 0),
        ('6', 0, 0),
        ('7', 0, 0),
        ('8', 0, 0),
        ('9', 0, 0),
        ('0', 0, 0),
        ('+', 0, 0),
        ('-', 0, 0),
        ('*', 0, 0),
        ('/', 0, 0),
        ('=', 1, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 15
[1]: 15
[2]: 15
[3]: 15
[4]: 15
[5]: 15
[6]: 15
[7]: 15
Max combinatory: 2562890625


In [44]:
var mini = new Nerdle()
#nullable enable
{
    Slot = new (char?, char[]?)[6]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('1', 0, 0),
        ('2', 0, 0),
        ('3', 0, 0),
        ('4', 0, 0),
        ('5', 0, 0),
        ('6', 0, 0),
        ('7', 0, 0),
        ('8', 0, 0),
        ('9', 0, 0),
        ('0', 0, 0),
        ('+', 0, 0),
        ('-', 0, 0),
        ('*', 0, 0),
        ('/', 0, 0),
        ('=', 1, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 11
[1]: 11
[2]: 11
[3]: 11
[4]: 11
[5]: 11
Max combinatory: 1771561


In [43]:
var micro = new Nerdle()
#nullable enable
{
    Slot = new (char?, char[]?)[5]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('1', 0, 0),
        ('2', 0, 0),
        ('3', 0, 0),
        ('4', 0, 0),
        ('5', 0, 0),
        ('6', 0, 0),
        ('7', 0, 0),
        ('8', 0, 0),
        ('9', 0, 0),
        ('0', 0, 0),
        ('+', 0, 0),
        ('-', 0, 0),
        ('*', 0, 0),
        ('/', 0, 0),
        ('=', 1, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 15
[1]: 15
[2]: 15
[3]: 15
[4]: 15
Max combinatory: 759375


In [42]:
var maxi = new Nerdle() // first input = 12*78=0936
#nullable enable
{
    Slot = new (char?, char[]?)[10]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('1', 0, 0),
        ('2', 0, 0),
        ('3', 0, 0),
        ('4', 0, 0),
        ('5', 0, 0),
        ('6', 0, 0),
        ('7', 0, 0),
        ('8', 0, 0),
        ('9', 0, 0),
        ('0', 0, 0),
        ('+', 0, 0),
        ('-', 0, 0),
        ('*', 0, 0),
        ('/', 0, 0),
        ('(', 0, 0),
        (')', 0, 0),
        ('²', 0, 0),
        ('³', 0, 0),
        ('=', 1, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 11
[1]: 11
[2]: 11
[3]: 11
[4]: 11
[5]: 11
[6]: 11
[7]: 11
[8]: 11
[9]: 11
Max combinatory: 25937424601


In [15]:
var wordle = new Nerdle()
#nullable enable
{
    Slot = new (char?, char[]?)[5]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('A', 0, 0),
        ('B', 0, 0),
        ('C', 0, 0),
        ('D', 0, 0),
        ('E', 0, 0),
        ('F', 0, 0),
        ('G', 0, 0),
        ('H', 0, 0),
        ('I', 0, 0),
        ('J', 0, 0),
        ('K', 0, 0),
        ('L', 0, 0),
        ('M', 0, 0),
        ('N', 0, 0),
        ('O', 0, 0),
        ('P', 0, 0),
        ('Q', 0, 0),
        ('R', 0, 0),
        ('S', 0, 0),
        ('T', 0, 0),
        ('U', 0, 0),
        ('V', 0, 0),
        ('W', 0, 0),
        ('X', 0, 0),
        ('Y', 0, 0),
        ('Z', 0, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 26
[1]: 26
[2]: 26
[3]: 26
[4]: 26
Max combinatory: 11881376


In [19]:
var cloudle = new Nerdle()
#nullable enable
{
    Slot = new (char?, char[]?)[5]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('0', 0, 0), // ☀️
        ('1', 0, 0), // 🌤️
        ('2', 0, 0), // 🌥️
        ('3', 0, 0), // ☁️
        ('4', 0, 0), // 🌦️
        ('5', 0, 0), // 🌧️
        ('6', 0, 0), // ⛈️
        ('7', 0, 0), // 🌨️
        ('8', 0, 0), // 🌫️
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 9
[1]: 9
[2]: 9
[3]: 9
[4]: 9
Max combinatory: 59049


In [ ]:
foreach (var line in maxi)
    Console.WriteLine(line);

In [48]:
var nerdle = new Nerdle()
#nullable enable
{
    Slot = new (char?, char[]?)[8]
    {
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
        (null, "".ToCharArray()),
    },
    Symbols = new (char c, int qty, int min)[]
    {
        ('1', 0, 0),
        ('2', 0, 0),
        ('3', 0, 0),
        ('4', 0, 0),
        ('5', 0, 0),
        ('6', 0, 0),
        ('7', 0, 0),
        ('8', 0, 0),
        ('9', 0, 0),
        ('0', 0, 0),
        ('+', 0, 0),
        ('-', 0, 0),
        ('*', 0, 0),
        ('/', 0, 0),

        ('(', 0, 0),
        (')', 0, 0),
        ('²', 0, 0),
        ('³', 0, 0),
        ('=', 1, 0),
    },
}.GetAllLines(printMaxCombinatory: true);

[0]: 19
[1]: 19
[2]: 19
[3]: 19
[4]: 19
[5]: 19
[6]: 19
[7]: 19
Max combinatory: 16983563041


In [14]:
foreach (var line in nerdle)
    Console.WriteLine(line);